In [1]:
from hvec_importers.rws import communicators as com
from hvec_importers.rws import rws
from hvec_importers.rws import helpers as hlp
from hvec_importers.rws import parsers as parse
import dateutil
import pandas as pd
import requests
import warnings

Understanding the raw data structure of data imported from RWS Waterinfo. Goal is to optimise the parsing procedure. The one in ddlpy relies on nested for-loops and is therefore very slow.

Get some data first. A deep dive in the library is necessary to obtain some raw data to experiment on. Do not try this at home!

In [2]:
warnings.simplefilter('ignore')

In [3]:
locations = rws.station_list()

In [4]:
sel = hlp.create_selection_table(
    locations = locations,
    name = 'Vlissingen', quantity = 'WATHTE')

In [5]:
sel

,Code,Locatie_MessageID,Coordinatenstelsel,X,Y,Naam,Parameter_Wat_Omschrijving,Compartiment.Code,Compartiment.Omschrijving,Eenheid.Code,Eenheid.Omschrijving,Grootheid.Code,Grootheid.Omschrijving,Hoedanigheid.Code,Hoedanigheid.Omschrijving,Parameter.Code,Parameter.Omschrijving,start,end
0,VLIS,19305,25831,541518.745920,5.699255e+06,Vlissingen,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,OW,Oppervlaktewater,cm,centimeter,WATHTE,Waterhoogte,NAP,t.o.v. Normaal Amsterdams Peil,NVT,Waarde is niet van toepassing,18000101,21001231
1,VLISSGN,20536,25831,541425.983215,5.699182e+06,Vlissingen,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,OW,Oppervlaktewater,cm,centimeter,WATHTE,Waterhoogte,NAP,t.o.v. Normaal Amsterdams Peil,NVT,Waarde is niet van toepassing,18000101,21001231


In [6]:
code = 'VLISSGN'
sel = sel.query('Code == @code')

In [7]:
start = dateutil.parser.parse('1943-01-31')
end = dateutil.parser.parse('1943-02-3')

In [8]:
req = hlp.create_data_request(sel, start, end)

In [9]:
req.keys()

dict_keys(['AquoPlusWaarnemingMetadata', 'Locatie', 'Periode'])

In [10]:
session = requests.session()
res = com.assert_data_available(sel, start, end, session)
res

True

In [11]:
raw = com._get_raw_slice(sel, start, end, session)
session.close()

In [12]:
raw

{'WaarnemingenLijst': [{'Locatie': {'Locatie_MessageID': 20536,
    'Coordinatenstelsel': '25831',
    'X': 541425.983214885,
    'Y': 5699181.90968435,
    'Naam': 'Vlissingen',
    'Code': 'VLISSGN'},
   'MetingenLijst': [{'Tijdstip': '1943-01-31T02:40:00.000+01:00',
     'Meetwaarde': {'Waarde_Numeriek': -135.0},
     'WaarnemingMetadata': {'StatuswaardeLijst': ['Gecontroleerd'],
      'BemonsteringshoogteLijst': ['-999999999'],
      'ReferentievlakLijst': ['NVT'],
      'OpdrachtgevendeInstantieLijst': ['RIKZMON_WAT'],
      'KwaliteitswaardecodeLijst': ['00']}},
    {'Tijdstip': '1943-01-31T05:40:00.000+01:00',
     'Meetwaarde': {'Waarde_Numeriek': -35.0},
     'WaarnemingMetadata': {'StatuswaardeLijst': ['Gecontroleerd'],
      'BemonsteringshoogteLijst': ['-999999999'],
      'ReferentievlakLijst': ['NVT'],
      'OpdrachtgevendeInstantieLijst': ['RIKZMON_WAT'],
      'KwaliteitswaardecodeLijst': ['00']}},
    {'Tijdstip': '1943-01-31T08:40:00.000+01:00',
     'Meetwaarde': {'

In [13]:
raw.keys()

dict_keys(['WaarnemingenLijst', 'Succesvol'])

In [23]:
raw

{'WaarnemingenLijst': [{'Locatie': {'Locatie_MessageID': 20536,
    'Coordinatenstelsel': '25831',
    'X': 541425.983214885,
    'Y': 5699181.90968435,
    'Naam': 'Vlissingen',
    'Code': 'VLISSGN'},
   'MetingenLijst': [{'Tijdstip': '1943-01-31T02:40:00.000+01:00',
     'Meetwaarde': {'Waarde_Numeriek': -135.0},
     'WaarnemingMetadata': {'StatuswaardeLijst': ['Gecontroleerd'],
      'BemonsteringshoogteLijst': ['-999999999'],
      'ReferentievlakLijst': ['NVT'],
      'OpdrachtgevendeInstantieLijst': ['RIKZMON_WAT'],
      'KwaliteitswaardecodeLijst': ['00']}},
    {'Tijdstip': '1943-01-31T05:40:00.000+01:00',
     'Meetwaarde': {'Waarde_Numeriek': -35.0},
     'WaarnemingMetadata': {'StatuswaardeLijst': ['Gecontroleerd'],
      'BemonsteringshoogteLijst': ['-999999999'],
      'ReferentievlakLijst': ['NVT'],
      'OpdrachtgevendeInstantieLijst': ['RIKZMON_WAT'],
      'KwaliteitswaardecodeLijst': ['00']}},
    {'Tijdstip': '1943-01-31T08:40:00.000+01:00',
     'Meetwaarde': {'

And here we are. "raw" is a dictionary with two entries. The second is a boolean, indicating if the import was succesfull. 

The good stuff is in "WaarnemingenLijst" but is disturbingly deep nested. To begin with, the value under the key is a list.

Tried a few json flatteners to no avail. Either do not work or give incorrect data. Try sequence of boring pandas methods.

In [14]:
WaarnemingenLijst = pd.json_normalize(raw, record_path = 'WaarnemingenLijst')
WaarnemingenLijst

,MetingenLijst,Locatie.Locatie_MessageID,Locatie.Coordinatenstelsel,Locatie.X,Locatie.Y,Locatie.Naam,Locatie.Code,AquoMetadata.AquoMetadata_MessageID,AquoMetadata.Parameter_Wat_Omschrijving,AquoMetadata.BemonsteringsApparaat.Code,...,AquoMetadata.Typering.Code,AquoMetadata.Typering.Omschrijving,AquoMetadata.Groepering.Code,AquoMetadata.Groepering.Omschrijving,AquoMetadata.WaardeBepalingstechniek.Code,AquoMetadata.WaardeBepalingstechniek.Omschrijving,AquoMetadata.WaardeBepalingsmethode.Code,AquoMetadata.WaardeBepalingsmethode.Omschrijving,AquoMetadata.WaardeBewerkingsmethode.Code,AquoMetadata.WaardeBewerkingsmethode.Omschrijving
0,"[{'Tijdstip': '1943-01-31T02:40:00.000+01:00',...",20536,25831,541425.983215,5.699182e+06,Vlissingen,VLISSGN,173945,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,NVT,...,NVT,Waarde is niet van toepassing,NVT,Niet van toepassing,NVT,Waarde is niet van toepassing,other:F009,Visuele aflezing van blad,NVT,Niet van toepassing


In [15]:
WaarnemingenLijst.columns

Index(['MetingenLijst', 'Locatie.Locatie_MessageID',
       'Locatie.Coordinatenstelsel', 'Locatie.X', 'Locatie.Y', 'Locatie.Naam',
       'Locatie.Code', 'AquoMetadata.AquoMetadata_MessageID',
       'AquoMetadata.Parameter_Wat_Omschrijving',
       'AquoMetadata.BemonsteringsApparaat.Code',
       'AquoMetadata.BemonsteringsApparaat.Omschrijving',
       'AquoMetadata.BemonsteringsMethode.Code',
       'AquoMetadata.BemonsteringsMethode.Omschrijving',
       'AquoMetadata.BemonsteringsSoort.Code',
       'AquoMetadata.BemonsteringsSoort.Omschrijving',
       'AquoMetadata.BioTaxon.Code', 'AquoMetadata.BioTaxon.Omschrijving',
       'AquoMetadata.BioTaxonType', 'AquoMetadata.BioTaxon_Compartiment.Code',
       'AquoMetadata.BioTaxon_Compartiment.Omschrijving',
       'AquoMetadata.Compartiment.Code',
       'AquoMetadata.Compartiment.Omschrijving', 'AquoMetadata.Eenheid.Code',
       'AquoMetadata.Eenheid.Omschrijving', 'AquoMetadata.Grootheid.Code',
       'AquoMetadata.Grootheid.Oms

In [17]:
MetingenLijst = pd.json_normalize(data = raw['WaarnemingenLijst'], record_path = 'MetingenLijst', meta = ['Locatie', 'AquoMetadata'])
MetingenLijst.columns

Index(['Tijdstip', 'Meetwaarde.Waarde_Numeriek',
       'WaarnemingMetadata.StatuswaardeLijst',
       'WaarnemingMetadata.BemonsteringshoogteLijst',
       'WaarnemingMetadata.ReferentievlakLijst',
       'WaarnemingMetadata.OpdrachtgevendeInstantieLijst',
       'WaarnemingMetadata.KwaliteitswaardecodeLijst', 'Locatie',
       'AquoMetadata'],
      dtype='object')

That's better. Some columns are lists of dictionaries. So flattening and keeping what we need only. Starting with location data

In [18]:
MetingenLijst['WaarnemingMetadata.StatuswaardeLijst'].explode()

0     Gecontroleerd
1     Gecontroleerd
2     Gecontroleerd
3     Gecontroleerd
4     Gecontroleerd
5     Gecontroleerd
6     Gecontroleerd
7     Gecontroleerd
8     Gecontroleerd
9     Gecontroleerd
10    Gecontroleerd
11    Gecontroleerd
12    Gecontroleerd
13    Gecontroleerd
14    Gecontroleerd
15    Gecontroleerd
16    Gecontroleerd
17    Gecontroleerd
18    Gecontroleerd
19    Gecontroleerd
20    Gecontroleerd
21    Gecontroleerd
22    Gecontroleerd
23    Gecontroleerd
Name: WaarnemingMetadata.StatuswaardeLijst, dtype: object

In [19]:
LocatieLijst = MetingenLijst['Locatie'].apply(pd.Series)
LocatieLijst.drop(columns = 'Locatie_MessageID', inplace = True)
LocatieLijst

,Coordinatenstelsel,X,Y,Naam,Code
0,25831,541425.983215,5.699182e+06,Vlissingen,VLISSGN
1,25831,541425.983215,5.699182e+06,Vlissingen,VLISSGN
2,25831,541425.983215,5.699182e+06,Vlissingen,VLISSGN
3,25831,541425.983215,5.699182e+06,Vlissingen,VLISSGN
4,25831,541425.983215,5.699182e+06,Vlissingen,VLISSGN
5,25831,541425.983215,5.699182e+06,Vlissingen,VLISSGN
6,25831,541425.983215,5.699182e+06,Vlissingen,VLISSGN
7,25831,541425.983215,5.699182e+06,Vlissingen,VLISSGN
8,25831,541425.983215,5.699182e+06,Vlissingen,VLISSGN
9,25831,541425.983215,5.699182e+06,Vlissingen,VLISSGN


In [20]:
LocatieLijst

,Coordinatenstelsel,X,Y,Naam,Code
0,25831,541425.983215,5.699182e+06,Vlissingen,VLISSGN
1,25831,541425.983215,5.699182e+06,Vlissingen,VLISSGN
2,25831,541425.983215,5.699182e+06,Vlissingen,VLISSGN
3,25831,541425.983215,5.699182e+06,Vlissingen,VLISSGN
4,25831,541425.983215,5.699182e+06,Vlissingen,VLISSGN
5,25831,541425.983215,5.699182e+06,Vlissingen,VLISSGN
6,25831,541425.983215,5.699182e+06,Vlissingen,VLISSGN
7,25831,541425.983215,5.699182e+06,Vlissingen,VLISSGN
8,25831,541425.983215,5.699182e+06,Vlissingen,VLISSGN
9,25831,541425.983215,5.699182e+06,Vlissingen,VLISSGN


Still a bit clunky, but way better than nested for-loops

In [21]:
meta = MetingenLijst['AquoMetadata'].apply(pd.Series)

In [22]:
meta.columns

Index(['AquoMetadata_MessageID', 'Parameter_Wat_Omschrijving',
       'BemonsteringsApparaat', 'BemonsteringsMethode', 'BemonsteringsSoort',
       'BioTaxon', 'BioTaxonType', 'BioTaxon_Compartiment', 'Compartiment',
       'Eenheid', 'Grootheid', 'Hoedanigheid', 'MeetApparaat',
       'MonsterBewerkingsMethode', 'Orgaan', 'Parameter',
       'PlaatsBepalingsApparaat', 'Typering', 'Groepering',
       'WaardeBepalingstechniek', 'WaardeBepalingsmethode',
       'WaardeBewerkingsmethode'],
      dtype='object')

In [ ]:
keep = ['Parameter_Wat_Omschrijving', 'Eenheid', 'MeetApparaat']
meta = meta[keep]

In [ ]:
meta

,Parameter_Wat_Omschrijving,Eenheid,MeetApparaat
0,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,"{'Code': 'cm', 'Omschrijving': 'centimeter'}","{'Code': '127', 'Omschrijving': 'Vlotter'}"
1,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,"{'Code': 'cm', 'Omschrijving': 'centimeter'}","{'Code': '127', 'Omschrijving': 'Vlotter'}"
2,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,"{'Code': 'cm', 'Omschrijving': 'centimeter'}","{'Code': '127', 'Omschrijving': 'Vlotter'}"
3,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,"{'Code': 'cm', 'Omschrijving': 'centimeter'}","{'Code': '127', 'Omschrijving': 'Vlotter'}"
4,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,"{'Code': 'cm', 'Omschrijving': 'centimeter'}","{'Code': '127', 'Omschrijving': 'Vlotter'}"
5,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,"{'Code': 'cm', 'Omschrijving': 'centimeter'}","{'Code': '127', 'Omschrijving': 'Vlotter'}"
6,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,"{'Code': 'cm', 'Omschrijving': 'centimeter'}","{'Code': '127', 'Omschrijving': 'Vlotter'}"
7,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,"{'Code': 'cm', 'Omschrijving': 'centimeter'}","{'Code': '127', 'Omschrijving': 'Vlotter'}"
8,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,"{'Code': 'cm', 'Omschrijving': 'centimeter'}","{'Code': '127', 'Omschrijving': 'Vlotter'}"
9,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,"{'Code': 'cm', 'Omschrijving': 'centimeter'}","{'Code': '127', 'Omschrijving': 'Vlotter'}"


In [ ]:
Eenheid = meta['Eenheid'].apply(pd.Series)['Code']

In [ ]:
meta.drop(columns = 'Eenheid', inplace = True)

In [ ]:
meta['Eenheid'] = Eenheid

In [ ]:
meta

,Parameter_Wat_Omschrijving,MeetApparaat,Eenheid
0,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,"{'Code': '127', 'Omschrijving': 'Vlotter'}",cm
1,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,"{'Code': '127', 'Omschrijving': 'Vlotter'}",cm
2,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,"{'Code': '127', 'Omschrijving': 'Vlotter'}",cm
3,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,"{'Code': '127', 'Omschrijving': 'Vlotter'}",cm
4,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,"{'Code': '127', 'Omschrijving': 'Vlotter'}",cm
5,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,"{'Code': '127', 'Omschrijving': 'Vlotter'}",cm
6,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,"{'Code': '127', 'Omschrijving': 'Vlotter'}",cm
7,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,"{'Code': '127', 'Omschrijving': 'Vlotter'}",cm
8,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,"{'Code': '127', 'Omschrijving': 'Vlotter'}",cm
9,Waterhoogte Oppervlaktewater t.o.v. Normaal Am...,"{'Code': '127', 'Omschrijving': 'Vlotter'}",cm
